### Bernoulli Naive-Bayes Model

In [124]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=4, suppress=True)

In [125]:
np.random.seed(0)

data = np.random.rand(200,5)
data = np.round(data, decimals=0)
X_train = data[:150, :-1]
y_train = data[:150, -1]

X_test = data[150:, :-1]
y_test = data[150:, -1]

In [126]:
X_test.shape, y_test.shape

((50, 4), (50,))

Base Model

In [127]:
from sklearn.naive_bayes import BernoulliNB
gnb = BernoulliNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [128]:
np.sum(y_pred == y_test) / len(y_test)

0.48

In [129]:
class Naive_Bayes:
    """Implements Naive-Bayes with Laplace Smoothing"""
    def compute_Px1y1(self, X, y):
        return (np.sum(X[y==1] == 1) + 1) / (np.sum(y==1) + self.d)
    
    def compute_Px1y0(self, X, y):
        return (np.sum(X[y==0] == 1) + 1) / (np.sum(y==0) + self.d)
    
    def compute_Px0y0(self, X, y):
        return (np.sum(X[y==0] == 0) + 1) / (np.sum(y==0) + self.d)
    
    def compute_Px0y1(self, X, y):
        return (np.sum(X[y==1] == 0) + 1) / (np.sum(y==1) + self.d)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    def compute_Py0(self, y):
        n = len(y)
        return (1/n) * np.sum(y==0)
    
    def fit(self, X, y):
        self.d = X.shape[1]
        PhiXy_arr = []
        for idx in range(self.d):
            PhiXy = np.zeros((2,2))
            PhiXy[1,1] = self.compute_Px1y1(X[:, idx], y)
            PhiXy[1,0] = self.compute_Px1y0(X[:, idx], y)
            PhiXy[0,0] =  self.compute_Px0y0(X[:, idx], y)
            PhiXy[0,1] =  self.compute_Px0y1(X[:, idx], y)
            
            PhiXy_arr.append(PhiXy)
        PhiXy_arr = np.array(PhiXy_arr)
        
        self.phiXy = PhiXy_arr                  
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = self.compute_Py0(y)

    def predict_proba(self, X):
        Pxy0 = np.zeros(X.shape)
        Pxy1 = np.zeros(X.shape)
        n, d = X.shape
        
        assert(d == self.d) # Ensure no of features given is same as that trained
        
        for row in range(n):
            for col in range(d):
                if X[row, col] == 1:
                    Pxy0[row, col] = self.phiXy[col][1][0]
                    Pxy1[row, col] = self.phiXy[col][1][1]
                else:
                    Pxy0[row, col] = self.phiXy[col][0][0]
                    Pxy1[row, col] = self.phiXy[col][0][1]
        
        Pxy1_i = np.multiply.reduce(Pxy1, axis=1)
        Pxy0_i = np.multiply.reduce(Pxy0, axis=1)
        
        Py0 = Pxy0_i * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Py1 = Pxy1_i * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [130]:
nb = Naive_Bayes()

In [131]:
nb.fit(X_train, y_train)

In [132]:
predictions = nb.predict(X_test)

In [133]:
predictions

array([1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1])

In [134]:
np.sum(predictions == y_test) / len(y_test)

0.48

In [135]:
nb.predict_proba(X_test)[0:5]

array([[0.0327, 0.0347],
       [0.0225, 0.0216],
       [0.0225, 0.0216],
       [0.0327, 0.0216],
       [0.0225, 0.0216]])